In [87]:
import pandas as pd
import numpy as np
import  datetime 

import io

In [88]:
# Change this variable to read in another file as long as it's the same path
filename = 'Sep_04_2019_2264098_0183.log'

base = './input/'
base_out = './output/'
with open(base+ filename, 'r') as f:
    data = f.read()
    
output_file = filename.split('.')[0] + '.sof'
print('Will output results to', output_file)

Will output results to Sep_04_2019_2264098_0183.sof


In [89]:
data = data.split('\n')
data = [x for x in data if x != '']

In [90]:
#count = [len(x.split(',')) for x in data]

In [91]:
data = [x.replace(': <-', ',') for x in data]

In [92]:
#keywords = [x.split(',')[1] for x in data]

In [93]:
#eywords[0:10]

In [94]:
#pd.Series.value_counts(keywords)

# Guide to Airmar codes
Taken from here:
http://www.airmartechnology.com/uploads/installguide/PB100TechnicalManual_rev1.007.pdf

    HCHDG Heading, Deviation and Variation 
    WIMWV Wind Speed and Angle, in relation to the vessel’s bow/centerline (relative) 
    YXXDR (type A) Transducer Measurements: wind chill, heat index, and station pressure 
    WIMDA Meteorological Composite. Barometric pressure, air temperature, relative humidity, dew point, wind direction, wind speed 
    TIROT    
    GPGSV GNSS Satellites in View 
    GPZDA Time and Date
    
    GPRMC Recommended Minimum Specific GNSS Data 
    GPGGA GPS Fix Data 

also see http://aprs.gids.nl/nmea/

In [95]:
GPZDA = [x for x in data if x.split(',')[1].strip()=='$GPZDA']

In [96]:
#GPZDA[0:10]

In [97]:
#GPZDA[0].split(',')

In [98]:
date = '/'.join(GPZDA[0].split(',')[3:6])

In [99]:
wind_data =[]
for line in data:
    if line.split(',')[1].strip()=='$WIMWV':
        line = line +','+ date
        wind_data.append(line)
    elif line.split(',')[1].strip()=='$GPZDA':
        date = '/'.join(line.split(',')[3:6])


In [100]:
#wind_data[0:10]

In [101]:
df = pd.read_csv(io.StringIO('\n'.join(wind_data)), names = ['Time', 'name', 'Wind Direction', 'Reference', 'Wind Speed', 'Units', 'Status', 'Date'])

In [102]:
df.head()

,Time,name,Wind Direction,Reference,Wind Speed,Units,Status,Date
0,14:10:04.481,$WIMWV,101.7,R,1.3,N,A*26,04/09/2019
1,14:10:04.961,$WIMWV,86.8,T,1.2,N,A*10,04/09/2019
2,14:10:27.675,$WIMWV,50.2,R,0.7,N,A*13,04/09/2019
3,14:10:28.059,$WIMWV,53.6,R,0.8,N,A*1B,04/09/2019
4,14:10:28.827,$WIMWV,33.9,R,0.7,N,A*1D,04/09/2019


In [103]:
df['Wind Speed'] = df['Wind Speed']  / 1.94384

In [104]:
df.head()

,Time,name,Wind Direction,Reference,Wind Speed,Units,Status,Date
0,14:10:04.481,$WIMWV,101.7,R,0.668779,N,A*26,04/09/2019
1,14:10:04.961,$WIMWV,86.8,T,0.617335,N,A*10,04/09/2019
2,14:10:27.675,$WIMWV,50.2,R,0.360112,N,A*13,04/09/2019
3,14:10:28.059,$WIMWV,53.6,R,0.411557,N,A*1B,04/09/2019
4,14:10:28.827,$WIMWV,33.9,R,0.360112,N,A*1D,04/09/2019


In [105]:
df.tail()

,Time,name,Wind Direction,Reference,Wind Speed,Units,Status,Date
66037,23:59:57.446,$WIMWV,252.0,R,0.771668,N,A*22,05/09/2019
66038,23:59:58.069,$WIMWV,250.9,R,0.771668,N,A*29,05/09/2019
66039,23:59:58.453,$WIMWV,250.0,R,0.771668,N,A*20,05/09/2019
66040,23:59:58.948,$WIMWV,248.5,R,0.771668,N,A*2C,05/09/2019
66041,23:59:59.587,$WIMWV,248.8,R,0.771668,N,A*21,05/09/2019


In [106]:

output_header = '%100,year,dayOfYear,HHMM,Sec,Windspeed,WindDir'

In [107]:

with open(base_out + output_file, 'w') as f:
    f.write(output_header + '\n')
    for i, row in df.iterrows():
        # Note that airmar date is day/month/year in this format
        date = datetime.datetime.strptime( row['Date']+' '+row['Time'].split('.')[0],  '%d/%m/%Y %H:%M:%S').timetuple()
        nextline = ','.join(['100' , str(date.tm_year),  str(date.tm_yday), '{:02d}{:02d}'.format(date.tm_hour,date.tm_min), str(date.tm_sec), str(row['Wind Speed']),str(row['Wind Direction'] )])
        f.write(nextline + '\n')
 
    